In [1]:
import sys
sys.path.append('../')
from ftx_utilities import *
%matplotlib inline

## benchmark

In [47]:
benchmark=pd.read_pickle('archived data/benchmark_blind.pickle')

In [61]:
benchmark=pd.read_pickle('archived data/full_run.pickle')
concentration_limit=0.5
slippage_override=2e-4
benchmark=benchmark[(benchmark['concentration_limit']==concentration_limit)&(benchmark['slippage_override']==slippage_override)]
benchmark

field,name,spotCarry,CarryEstimator,optimalWeight,ExpectedCarry,RealizedCarry,excessIM,excessMM,transactionCost,absWeight,time,slippage_override,concentration_limit,signal_horizon,holding_period
719076,KNC-PERP,-0.202056,0.224366,-5.000000e+04,-1.121829e+04,2.293770e+04,-7.020443e+04,-7.767821e+04,-9.308166e+03,5.000000e+04,2021-07-01 18:00:00,0.0002,0.5,0 days 12:00:00,0 days 06:00:00
719077,CEL-PERP,0.376938,0.311149,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-5.404286e+04,0.000000e+00,2021-07-01 18:00:00,0.0002,0.5,0 days 12:00:00,0 days 06:00:00
719078,MKR-PERP,-0.070128,-0.151250,-1.839735e-22,2.782600e-23,3.010395e-23,-2.472820e-22,-2.284518e-22,-2.862253e+04,1.839735e-22,2021-07-01 18:00:00,0.0002,0.5,0 days 12:00:00,0 days 06:00:00
719079,1INCH-PERP,-0.201618,-0.187709,0.000000e+00,-0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,-3.514053e+04,0.000000e+00,2021-07-01 18:00:00,0.0002,0.5,0 days 12:00:00,0 days 06:00:00
719080,SNX-PERP,0.140256,-0.097157,3.242977e+04,-3.150763e+03,1.895196e+03,2.531163e+04,2.134863e+04,-1.179609e-07,3.242977e+04,2021-07-01 18:00:00,0.0002,0.5,0 days 12:00:00,0 days 06:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078268,XRP-PERP,-0.070040,0.102918,1.584509e+04,1.630738e+03,1.194292e+02,1.426058e+04,1.457748e+04,-5.624697e+00,1.584509e+04,2021-09-24 00:00:00,0.0002,0.5,30 days 00:00:00,5 days 00:00:00
1078269,TRX-PERP,-0.236594,0.114632,4.018698e+02,4.606709e+01,-1.447280e+01,3.415893e+02,3.496267e+02,-4.865116e+00,4.018698e+02,2021-09-24 00:00:00,0.0002,0.5,30 days 00:00:00,5 days 00:00:00
1078270,BNB-PERP,-0.239136,0.119973,2.781063e+01,3.336523e+00,5.390761e-01,2.502957e+01,2.558578e+01,-8.126266e-01,2.781063e+01,2021-09-24 00:00:00,0.0002,0.5,30 days 00:00:00,5 days 00:00:00
1078271,USD,0.070040,0.054393,-2.162064e+05,-1.176020e+04,-1.314193e+04,-2.378270e+05,-2.226926e+05,0.000000e+00,2.162064e+05,2021-09-24 00:00:00,0.0002,0.5,30 days 00:00:00,5 days 00:00:00


In [64]:
benchmark['totalPnL']=(benchmark['RealizedCarry']+benchmark['transactionCost'])
benchmark['trackingError']=benchmark['RealizedCarry']-benchmark['ExpectedCarry']
perfs=benchmark.groupby(by=['name'])['absWeight'].describe().sort_values(by='mean',key=abs,ascending=False).head(20)/1e5
mains=benchmark[benchmark['name'].isin(list(perfs.index)+['total','USD'])]

In [66]:
(mains.pivot(index='time',columns='name',values='RealizedCarry').sum()/24/365.25).iplot()

ValueError: Index contains duplicate entries, cannot reshape

In [41]:
stats=mains.groupby(by=['name'])[['RealizedCarry','transactionCost','trackingError','optimalWeight']].agg(['mean','std'])*100/1e5
stats[('error','')]=np.sqrt((np.power(stats[('trackingError','mean')],2)+np.power(stats[('trackingError','std')],2))/(np.power(stats[('RealizedCarry','mean')],2)+np.power(stats[('RealizedCarry','std')],2)))
stats

field      RealizedCarry            transactionCost             trackingError  \
                    mean        std            mean         std          mean   
name                                                                            
AAVE-PERP       3.421307   6.781093      -19.630977   34.830130     -0.556535   
ALPHA-PERP      6.612391   9.202989      -23.259970   38.014915     -0.639471   
BAND-PERP       5.123217   8.938919      -17.190771   28.545262     -0.347781   
BCH-PERP        2.002124   3.618912      -18.572621   34.291840     -0.368856   
BNT-PERP        7.526665  11.096281      -35.475382   45.639663     -1.687329   
CEL-PERP        4.697713   8.351736      -38.882663   55.403079     -2.333005   
DOGE-PERP       4.586500   8.051894      -16.903453   31.270699     -0.363114   
GRT-PERP        8.068580  10.816846      -21.233930   33.993033     -0.543631   
KNC-PERP        3.631510   7.753744      -21.874001   34.565270     -0.669080   
LINK-PERP       2.869481   5.782604      -14.613923   28.683138     -0.226603   
MKR-PERP        6.922685   8.796582      -19.791104   32.166179     -0.439694   
OKB-PERP       14.022361  16.810620      -22.723497   42.302906     -1.102332   
RUNE-PERP       6.124588   9.752879      -21.810663   35.413633     -0.649864   
SNX-PERP        4.180927   8.634974      -17.174119   31.607875     -0.478113   
SOL-PERP        2.732079   5.258952      -14.116719   26.966501     -0.253156   
SUSHI-PERP      4.693480   7.307900      -20.254219   31.841700     -0.368824   
SXP-PERP        5.153939  14.685830      -19.409154   33.267010     -0.480862   
UNI-PERP        3.235307   6.239048      -12.396933   23.823009     -0.141482   
USD            -6.690893  14.555804        0.000000    0.000000     -0.250589   
total         105.409644  48.316056     -504.508569  349.185739    -14.234859   

field                 optimalWeight                accuracy  
                  std          mean           std            
name                                                         
AAVE-PERP    3.981151     12.012818  1.937581e+01  0.529257  
ALPHA-PERP   4.272451     18.398399  2.439167e+01  0.381218  
BAND-PERP    2.857448      4.098270  2.187616e+01  0.279388  
BCH-PERP     2.065194     10.665971  2.102280e+01  0.507245  
BNT-PERP     7.547994    -18.249089  1.929092e+01  0.576836  
CEL-PERP     8.285114      3.728075  2.754214e+01  0.898255  
DOGE-PERP    3.440503     14.360477  2.181246e+01  0.373344  
GRT-PERP     5.081284     -9.514408  2.878132e+01  0.378689  
KNC-PERP     4.993420     -6.099548  1.973640e+01  0.588417  
LINK-PERP    1.729469     12.401287  1.890515e+01  0.270200  
MKR-PERP     3.059837      2.490145  2.983602e+01  0.276156  
OKB-PERP     8.853247    -11.250703  3.568008e+01  0.407544  
RUNE-PERP    4.534670      8.367129  2.467545e+01  0.397778  
SNX-PERP     3.324360      1.055612  2.136808e+01  0.350073  
SOL-PERP     2.749426     13.202565  1.836752e+01  0.465900  
SUSHI-PERP   2.448457     17.471978  2.086610e+01  0.285089  
SXP-PERP    15.302805     14.018829  2.077309e+01  0.983706  
UNI-PERP     1.866238      9.630834  1.970164e+01  0.266305  
USD          6.649631    -10.230468  1.835580e+02  0.415378  
total       31.816274    100.000000  3.458312e-14  0.300594

## todo
# - tx cost = 0 for benchmark, 5bps
# - realized vs expected fit -> finer hold/sig
# - raise the bar for volume, blowup
# - munraj

## Comparative run statistics

In [46]:
runs=compile_runs()
runs

""


In [43]:
c=0.5
all=1*a.xs([c,'RealizedCarry'],
       level=['concentration_limit','field'])\
    +a.xs([c,'transactionCost'],
       level=['concentration_limit','field'])
all['total'].groupby(by=['holding_period','signal_horizon']).describe()*100

C:\Users\david\AppData\Local\Temp/ipykernel_11608/3313751634.py:2: FutureWarning:

Passing lists as key for xs is deprecated and will be removed in a future version. Pass key as a tuple instead.



TypeError: Index must be a MultiIndex

## benchmark: transaction costs block rebalance ?

In [ ]:
benchmark=a.xs([c,timedelta(hours=1),timedelta(hours=1)],
       level=['concentration_limit','holding_period','signal_horizon']).reorder_levels(['field','time'])

## Zoom on a run

In [ ]:
run=a.xs([c,timedelta(days=2),timedelta(days=7)],
       level=['concentration_limit','holding_period','signal_horizon']).reorder_levels(['field','time'])

In [ ]:
relevant=(run.T['absWeight'].T).max().sort_values(ascending=False)
relevant=relevant[relevant>0.02]
run=run[relevant.index]

In [ ]:
fig=px.line(run.loc['optimalWeight'])
fig.show()

In [ ]:
fig=px.line(run.loc['RealizedCarry'])
fig.show()

In [ ]:
fig=px.line(run.loc['ExpectedCarry'])
fig.show()